In [267]:
#pandas libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.model_selection as modsel

import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Activation
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


%run "../notebooks/cargar_df.ipynb"

In [149]:
train = get_train()

In [150]:
test = get_test()

In [151]:
train.head().T

,0,1,2,3,4
ID,27761,27760,27446,16808,16805
Region,EMEA,EMEA,Americas,Americas,Americas
Territory,None,None,NW America,NW America,NW America
"Pricing, Delivery_Terms_Quote_Appr",1,0,0,1,1
"Pricing, Delivery_Terms_Approved",1,0,0,0,0
Bureaucratic_Code_0_Approval,1,0,0,1,1
Bureaucratic_Code_0_Approved,1,0,0,0,0
Bureaucratic_Code,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_5
Account_Created_Date,2015-06-16 00:00:00,2015-06-16 00:00:00,2015-04-21 00:00:00,2013-07-27 00:00:00,2013-07-27 00:00:00
Source,None,None,Source_7,Source_11,Source_11


In [152]:
test.head().T

,0,1,2,3,4
ID,6140,6146,6151,6118,6124
Region,EMEA,EMEA,EMEA,EMEA,EMEA
Territory,Germany,Germany,Germany,Germany,Germany
"Pricing, Delivery_Terms_Quote_Appr",1,1,1,1,1
"Pricing, Delivery_Terms_Approved",1,1,1,1,1
Bureaucratic_Code_0_Approval,1,1,1,1,1
Bureaucratic_Code_0_Approved,1,1,1,1,1
Bureaucratic_Code,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4,Bureaucratic_Code_4
Account_Created_Date,2017-07-05 00:00:00,2017-07-05 00:00:00,2017-07-05 00:00:00,2017-07-05 00:00:00,2017-07-05 00:00:00
Source,Source_7,Source_7,Source_7,Source_7,Source_7


In [153]:
train = train[train.Stage.isin(['Closed Won', 'Closed Lost'])].copy()
train.shape

(16883, 47)

In [154]:
train.Stage.value_counts()

Closed Won     9533
Closed Lost    7350
Name: Stage, dtype: int64

In [155]:
# Me quedo con los registros con la oportunidad terminada
train.Stage.replace({'Closed Won': 1, 'Closed Lost': 0}, inplace=True)

In [156]:
train.Stage.value_counts()

1    9533
0    7350
Name: Stage, dtype: int64

In [157]:
print('Cantidad de opportunity_id de train:',train.Opportunity_ID.nunique())
print('Cantidad de opportunity_id de test:',test.Opportunity_ID.nunique())

Cantidad de opportunity_id de train: 9791
Cantidad de opportunity_id de test: 1567


In [158]:
# Junto el set de train y test en un nuevo data frame data
data = pd.concat([train.drop('Stage', axis=1), test])
data

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount
0,27761,EMEA,None,1,1,1,1,Bureaucratic_Code_4,2015-06-16,None,...,2016-05-01,2016-06-30,2016-05-01,Q2,2016,10,EUR,5272800.0,EUR,5272800.0
1,27760,EMEA,None,0,0,0,0,Bureaucratic_Code_4,2015-06-16,None,...,2016-01-18,2016-01-20,2016-01-01,Q1,2016,0,EUR,48230.0,EUR,48230.0
2,27446,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,2016-01-25,2016-01-25,2016-01-01,Q1,2016,0,USD,83865.6,USD,83865.6
3,16808,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018-02-01,2018-03-31,2018-02-01,Q1,2018,14,USD,7421881.5,USD,7421881.5
4,16805,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,2018-02-01,2018-02-28,2018-02-01,Q1,2018,25,USD,13357192.5,USD,13357192.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,2017-06-12,Source_3,...,2019-05-01,2019-05-10,2019-05-01,Q2,2019,0,USD,124740.0,USD,147750.0
2547,15218,Americas,SE America,1,1,0,0,Bureaucratic_Code_4,2018-06-08,None,...,2019-10-01,2019-12-31,2019-10-01,Q4,2019,0,USD,45054.9,USD,45054.9
2548,15224,Americas,SE America,1,1,1,1,Bureaucratic_Code_4,2018-06-08,None,...,2019-10-01,2019-12-31,2019-10-01,Q4,2019,0,USD,100122.0,USD,100122.0
2549,7286,Americas,NE America,1,1,0,0,Bureaucratic_Code_4,2018-08-29,None,...,2019-08-01,2019-08-01,2019-08-01,Q3,2019,0,USD,143220.0,USD,143220.0


Busco registros que tengan la misma opportunity id para saber cuales son
las columnas que pueden cambiar el valor

In [159]:
b = data.groupby('Opportunity_ID').ID.count() > 1
b = b[b] # Dejo solo los trues
df = data[data.Opportunity_ID.isin(b.index)]
df

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount
8,24353,Japan,None,1,0,0,0,Bureaucratic_Code_5,2015-04-20,None,...,2016-02-29,2016-02-29,2016-02-01,Q1,2016,0,JPY,15600.00,JPY,470400.00
9,24355,Japan,None,1,0,0,0,Bureaucratic_Code_5,2015-04-20,None,...,2016-02-29,2016-02-29,2016-02-01,Q1,2016,0,JPY,4400.00,JPY,470400.00
10,24356,Japan,None,1,0,0,0,Bureaucratic_Code_5,2015-04-20,None,...,2016-02-29,2016-02-29,2016-02-01,Q1,2016,0,JPY,6200.00,JPY,470400.00
11,24357,Japan,None,1,0,0,0,Bureaucratic_Code_5,2015-04-20,None,...,2016-02-29,2016-02-29,2016-02-01,Q1,2016,0,JPY,140000.00,JPY,470400.00
12,24358,Japan,None,1,0,0,0,Bureaucratic_Code_5,2015-04-20,None,...,2016-02-29,2016-02-29,2016-02-01,Q1,2016,0,JPY,304200.00,JPY,470400.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,20767,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,2013-07-27,Source_11,...,2019-04-30,2019-04-30,2019-04-01,Q2,2019,1,USD,280714.20,USD,935714.00
2540,14085,EMEA,Norway,1,1,1,0,Bureaucratic_Code_2,2018-04-19,Source_9,...,2019-05-13,2019-05-17,2019-05-01,Q2,2019,0,EUR,45841.95,EUR,74834.55
2541,14091,EMEA,Norway,1,1,1,0,Bureaucratic_Code_2,2018-04-19,Source_9,...,2019-05-13,2019-05-17,2019-05-01,Q2,2019,0,EUR,28992.60,EUR,74834.55
2545,16339,EMEA,KSA,1,1,0,0,Bureaucratic_Code_4,2017-06-12,Source_3,...,2019-05-01,2019-05-10,2019-05-01,Q2,2019,0,USD,23010.00,USD,147750.00


In [160]:
print('Columnas que cambian de valor dependiendo de la Id')

size = df.Opportunity_ID.unique().size
diff_columns = []
for column in df.columns:
    if (df.groupby('Opportunity_ID')[column].value_counts().count() != size):
        diff_columns.append(column)
diff_columns

Columnas que cambian de valor dependiendo de la Id


['ID',
 'Product_Family',
 'Product_Name',
 'ASP',
 'ASP_(converted)',
 'Planned_Delivery_Start_Date',
 'Planned_Delivery_End_Date',
 'Month',
 'Delivery_Quarter',
 'Delivery_Year',
 'TRF',
 'Total_Amount']

In [161]:
data.columns

Index(['ID', 'Region', 'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Bureaucratic_Code',
       'Account_Created_Date', 'Source ', 'Billing_Country', 'Account_Name',
       'Opportunity_Name', 'Opportunity_ID', 'Sales_Contract_No',
       'Account_Owner', 'Opportunity_Owner', 'Account_Type',
       'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date',
       'Last_Modified_Date', 'Last_Modified_By', 'Product_Family',
       'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'TRF', 'Total_Amount_Currency',
       'Total_Amount', 'Total_Taxable_Amount_Currency',
       'Total_Taxable_Amount'],
      dtype='obje

In [162]:
grouped_data = data.groupby('Opportunity_ID').agg(\
    {'Region':'first',
     'Territory':'first',
     'Pricing, Delivery_Terms_Quote_Appr':'first',
     'Pricing, Delivery_Terms_Approved':'first',
     'Bureaucratic_Code_0_Approval':'first',
     'Bureaucratic_Code_0_Approved':'first',
     'Bureaucratic_Code':'first',
     'Account_Created_Date':'first',
     'Source ':'first',
     'Billing_Country':'first',
     'Account_Name':'first',
     'Opportunity_Name':'first',
     'Account_Owner':'first',
     'Opportunity_Owner':'first',
     'Account_Type':'first',
     'Opportunity_Type':'first',
     'Quote_Type':'first',
     'Delivery_Terms':'first',
     'Opportunity_Created_Date':'first',
     'Brand':'first',
     'Product_Type':'first',
     'Size':'first',
     'Product_Category_B':'first',
     'Price':'first',
     'Currency':'first',
     'Quote_Expiry_Date':'first',
     'Last_Modified_Date':'first',
     'Last_Modified_By':'first',
     'ASP_Currency':'first',
     'Total_Amount_Currency':'first',
     'Total_Taxable_Amount_Currency':'first',
     'Total_Taxable_Amount':'first'}).reset_index()
grouped_data

,Opportunity_ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Account_Created_Date,Source,...,Product_Category_B,Price,Currency,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,ASP_Currency,Total_Amount_Currency,Total_Taxable_Amount_Currency,Total_Taxable_Amount
0,0,EMEA,None,1,1,1,1,Bureaucratic_Code_4,2015-06-16,None,...,None,None,None,1/30/2016,2016-06-13,Person_Name_18,EUR,EUR,EUR,5272800.0
1,1,EMEA,None,0,0,0,0,Bureaucratic_Code_4,2015-06-16,None,...,None,None,None,1/20/2016,2016-01-15,Person_Name_20,EUR,EUR,EUR,48230.0
2,2,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,2015-04-21,Source_7,...,None,None,None,9/30/2016,2016-09-29,Person_Name_8,USD,USD,USD,83865.6
3,3,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,Product_Category_B_16,0.42,USD,NaT,2018-03-27,Person_Name_8,USD,USD,USD,7421881.5
4,4,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,2013-07-27,Source_11,...,Product_Category_B_16,0.42,USD,6/2/2017,2018-03-27,Person_Name_8,USD,USD,USD,13357192.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,EMEA,Norway,1,1,0,0,Bureaucratic_Code_4,2013-07-27,Source_9,...,None,None,None,4/3/2016,2016-04-01,Person_Name_13,EUR,EUR,EUR,401700.0
11354,12800,Americas,NE America,1,0,0,0,Bureaucratic_Code_5,2014-06-16,None,...,None,None,None,1/15/2016,2016-10-10,Person_Name_13,USD,USD,USD,21332500.0
11355,12801,EMEA,Austria,1,1,1,1,Bureaucratic_Code_4,2016-01-15,Source_7,...,None,None,None,NaT,2016-01-19,Person_Name_13,EUR,EUR,EUR,299715.0
11356,12802,Americas,NE America,1,1,1,1,Bureaucratic_Code_4,2015-10-20,None,...,None,None,None,2/29/2016,2016-01-22,Person_Name_3,USD,USD,USD,0.0


In [163]:
X = pd.DataFrame({'Opportunity_ID':grouped_data.Opportunity_ID})

## Agrego Features

In [164]:
X['Pricing, Delivery_Terms_Quote_Appr'] = grouped_data['Pricing, Delivery_Terms_Quote_Appr']

In [165]:
X['Pricing, Delivery_Terms_Approved'] = grouped_data['Pricing, Delivery_Terms_Approved']

In [166]:
pricing_ok = grouped_data.agg(lambda x: int(x['Pricing, Delivery_Terms_Quote_Appr'] \
              == x['Pricing, Delivery_Terms_Approved']), axis = 1)
pricing_ok 

0        1
1        1
2        1
3        0
4        0
        ..
11353    1
11354    0
11355    1
11356    1
11357    1
Length: 11358, dtype: int64

In [167]:
# Binaria, es 1 si se necesitaba aprobación y se aprobó o si no la necesitaba
# Es 0 si necesitaba aprobacion y no se aprobo
X['pricing_aprobada'] = pricing_ok

X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada
0,0,1,1,1
1,1,0,0,1
2,2,0,0,1
3,3,1,0,0
4,4,1,0,0


Agrego features binarios Bureaucratic_Code

In [168]:
X['Bureaucratic_Code_0_Approval'] = grouped_data['Bureaucratic_Code_0_Approval']

In [169]:
X['Bureaucratic_Code_0_Approved'] = grouped_data['Bureaucratic_Code_0_Approved']

In [170]:
bureaucratic_ok = grouped_data.agg(lambda x: int(x['Bureaucratic_Code_0_Approval'] \
              == x['Bureaucratic_Code_0_Approved']), axis = 1)
bureaucratic_ok 

0        1
1        1
2        1
3        0
4        0
        ..
11353    1
11354    1
11355    1
11356    1
11357    1
Length: 11358, dtype: int64

In [171]:
# Binaria, es 1 si se necesitaba aprobación y se aprobó o si no la necesitaba
# Si necesitaba aprobacion y no se aprobo es 0
X['bureaucratic_aprobada'] = bureaucratic_ok

X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada
0,0,1,1,1,1,1,1
1,1,0,0,1,0,0,1
2,2,0,0,1,0,0,1
3,3,1,0,0,1,0,0
4,4,1,0,0,1,0,0


Feature: binaria 1 si hay mas de 1 ID de la oportunidad, 0 si hay solo un registro

In [172]:
ids = data.groupby('Opportunity_ID').ID.count()
ids

Opportunity_ID
0        1
1        1
2        1
3        1
4        1
        ..
12799    1
12800    2
12801    3
12802    1
12803    1
Name: ID, Length: 11358, dtype: int64

In [173]:
group1 = (ids != 1).astype('int8').reset_index()
group1

,Opportunity_ID,ID
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
11353,12799,0
11354,12800,1
11355,12801,1
11356,12802,0


In [174]:
X['amount_ids'] = group1.ID
X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids
0,0,1,1,1,1,1,1,0
1,1,0,0,1,0,0,1,0
2,2,0,0,1,0,0,1,0
3,3,1,0,0,1,0,0,0
4,4,1,0,0,1,0,0,0


Feature: menor anio de creacion de oportunidad 

In [175]:
data.groupby('Opportunity_ID').Delivery_Year.min()

Opportunity_ID
0        2016
1        2016
2        2016
3        2018
4        2018
         ... 
12799    2016
12800    2016
12801    2016
12802    2016
12803    2016
Name: Delivery_Year, Length: 11358, dtype: int16

In [176]:
min_year = data.groupby('Opportunity_ID').Delivery_Year.min().reset_index()

In [177]:
X['min_year'] = min_year.Delivery_Year
X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year
0,0,1,1,1,1,1,1,0,2016
1,1,0,0,1,0,0,1,0,2016
2,2,0,0,1,0,0,1,0,2016
3,3,1,0,0,1,0,0,0,2018
4,4,1,0,0,1,0,0,0,2018


Feature: cantidad de dias activos

In [178]:
# Calculo la cantidad de dias activos que tuvo cada oportunidad
days = grouped_data['Last_Modified_Date'] - grouped_data['Opportunity_Created_Date']

In [179]:
days.dt.days

0        189
1         39
2        296
3        840
4        840
        ... 
11353    119
11354    311
11355     46
11356     48
11357    298
Length: 11358, dtype: int64

In [180]:
X['active_days'] = days.dt.days
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days
0,0,1,1,1,1,1,1,0,2016,189
1,1,0,0,1,0,0,1,0,2016,39
2,2,0,0,1,0,0,1,0,2016,296
3,3,1,0,0,1,0,0,0,2018,840
4,4,1,0,0,1,0,0,0,2018,840
...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119
11354,12800,1,0,0,0,0,1,1,2016,311
11355,12801,1,1,1,1,1,1,1,2016,46
11356,12802,1,1,1,1,1,1,0,2016,48


In [181]:
# Me fijo cuantos null hay por columnas
print('Cantidad de nulos por columna')
grouped_data.replace({'None':np.NaN}).isna().sum()

Cantidad de nulos por columna


Opportunity_ID                            0
Region                                    0
Territory                              1959
Pricing, Delivery_Terms_Quote_Appr        0
Pricing, Delivery_Terms_Approved          0
Bureaucratic_Code_0_Approval              0
Bureaucratic_Code_0_Approved              0
Bureaucratic_Code                         0
Account_Created_Date                      0
Source                                 5590
Billing_Country                          27
Account_Name                              0
Opportunity_Name                          0
Account_Owner                             0
Opportunity_Owner                         0
Account_Type                            104
Opportunity_Type                          0
Quote_Type                                0
Delivery_Terms                            0
Opportunity_Created_Date                  0
Brand                                 10449
Product_Type                          10467
Size                            

Paso las columnas con muchos nulos a binario:
    1 si tiene valor y 0 si es nulo

In [182]:
X['has_source'] = (grouped_data['Source '] == 'None').astype('int8')
X['has_brand'] = (grouped_data['Brand'] == 'None').astype('int8')
X['has_product_Type'] = (grouped_data['Product_Type'] == 'None').astype('int8')
X['has_size'] = (grouped_data['Size'] == 'None').astype('int8')
X['has_product_Category_B'] = (grouped_data['Product_Category_B'] == 'None').astype('int8')
X['has_price'] = (grouped_data['Price'] == 'None').astype('int8')
X['has_currency'] = (grouped_data['Currency'] == 'None').astype('int8')
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,has_source,has_brand,has_product_Type,has_size,has_product_Category_B,has_price,has_currency
0,0,1,1,1,1,1,1,0,2016,189,1,1,1,1,1,1,1
1,1,0,0,1,0,0,1,0,2016,39,1,1,1,1,1,1,1
2,2,0,0,1,0,0,1,0,2016,296,0,1,1,1,1,1,1
3,3,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0
4,4,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,0,1,1,1,1,1,1
11354,12800,1,0,0,0,0,1,1,2016,311,1,1,1,1,1,1,1
11355,12801,1,1,1,1,1,1,1,2016,46,0,1,1,1,1,1,1
11356,12802,1,1,1,1,1,1,0,2016,48,1,1,1,1,1,1,1


In [183]:
# Agrego feature, el numero de delivery quarter

first_quarter = data.groupby('Opportunity_ID').Delivery_Quarter.first()\
                     .agg(lambda x: x[1:2]).astype('int8')\
                     .reset_index().drop('Opportunity_ID', axis=1)

X = pd.concat([X, first_quarter], axis = 1, sort=False)
X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,has_source,has_brand,has_product_Type,has_size,has_product_Category_B,has_price,has_currency,Delivery_Quarter
0,0,1,1,1,1,1,1,0,2016,189,1,1,1,1,1,1,1,2
1,1,0,0,1,0,0,1,0,2016,39,1,1,1,1,1,1,1,1
2,2,0,0,1,0,0,1,0,2016,296,0,1,1,1,1,1,1,1
3,3,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0,1
4,4,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0,1


Features: menor mes y anio de la columna Month

In [184]:
data['month'] = data.Month.dt.month
data['year'] = data.Month.dt.year

In [185]:
u = data.groupby('Opportunity_ID').agg({'month':'min', 'year':'min'})\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X, u], axis = 1, sort=False)
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,has_source,has_brand,has_product_Type,has_size,has_product_Category_B,has_price,has_currency,Delivery_Quarter,month,year
0,0,1,1,1,1,1,1,0,2016,189,1,1,1,1,1,1,1,2,5,2016
1,1,0,0,1,0,0,1,0,2016,39,1,1,1,1,1,1,1,1,1,2016
2,2,0,0,1,0,0,1,0,2016,296,0,1,1,1,1,1,1,1,1,2016
3,3,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0,1,2,2018
4,4,1,0,0,1,0,0,0,2018,840,0,0,0,0,0,0,0,1,2,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,0,1,1,1,1,1,1,2,4,2016
11354,12800,1,0,0,0,0,1,1,2016,311,1,1,1,1,1,1,1,4,10,2016
11355,12801,1,1,1,1,1,1,1,2016,46,0,1,1,1,1,1,1,1,3,2016
11356,12802,1,1,1,1,1,1,0,2016,48,1,1,1,1,1,1,1,2,4,2016


In [186]:
# Paso region a one hot encoder

one_hot_enc = pd.get_dummies(grouped_data.Region)

X = pd.concat([X,one_hot_enc], axis = 1, sort=False)
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,has_price,has_currency,Delivery_Quarter,month,year,APAC,Americas,EMEA,Japan,Middle East
0,0,1,1,1,1,1,1,0,2016,189,...,1,1,2,5,2016,0,0,1,0,0
1,1,0,0,1,0,0,1,0,2016,39,...,1,1,1,1,2016,0,0,1,0,0
2,2,0,0,1,0,0,1,0,2016,296,...,1,1,1,1,2016,0,1,0,0,0
3,3,1,0,0,1,0,0,0,2018,840,...,0,0,1,2,2018,0,1,0,0,0
4,4,1,0,0,1,0,0,0,2018,840,...,0,0,1,2,2018,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,...,1,1,2,4,2016,0,0,1,0,0
11354,12800,1,0,0,0,0,1,1,2016,311,...,1,1,4,10,2016,0,1,0,0,0
11355,12801,1,1,1,1,1,1,1,2016,46,...,1,1,1,3,2016,0,0,1,0,0
11356,12802,1,1,1,1,1,1,0,2016,48,...,1,1,2,4,2016,0,1,0,0,0


Agrego feature de Bureaucratic code a one hot encoder

In [187]:
data.Bureaucratic_Code.value_counts()

Bureaucratic_Code_4    14544
Bureaucratic_Code_5     4153
Bureaucratic_Code_2      351
Bureaucratic_Code_1      303
Bureaucratic_Code_0       80
Bureaucratic_Code_3        2
Bureaucratic_Code_6        1
Name: Bureaucratic_Code, dtype: int64

In [188]:
one_hot_enc = pd.get_dummies(grouped_data.Bureaucratic_Code)

X = pd.concat([X,one_hot_enc], axis=1, sort=False)
X.head()

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,EMEA,Japan,Middle East,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,Bureaucratic_Code_3,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6
0,0,1,1,1,1,1,1,0,2016,189,...,1,0,0,0,0,0,0,1,0,0
1,1,0,0,1,0,0,1,0,2016,39,...,1,0,0,0,0,0,0,1,0,0
2,2,0,0,1,0,0,1,0,2016,296,...,0,0,0,0,0,0,0,1,0,0
3,3,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,0,0,0,0,1,0
4,4,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,0,0,0,0,1,0


Paso account type a one hot encoder

In [189]:
grouped_data['Account_Type'].value_counts()

Account_Type_2    5443
Account_Type_0    4014
Account_Type_5    1637
Account_Type_1     126
None               104
Account_Type_6      17
Account_Type_4      17
Name: Account_Type, dtype: int64

In [190]:
one_hot_enc = pd.get_dummies(grouped_data.Account_Type)

X = pd.concat([X,one_hot_enc], axis=1, sort=False)
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None
0,0,1,1,1,1,1,1,0,2016,189,...,1,0,0,0,0,1,0,0,0,0
1,1,0,0,1,0,0,1,0,2016,39,...,1,0,0,0,0,1,0,0,0,0
2,2,0,0,1,0,0,1,0,2016,296,...,1,0,0,0,0,0,0,1,0,0
3,3,1,0,0,1,0,0,0,2018,840,...,0,1,0,0,0,0,0,1,0,0
4,4,1,0,0,1,0,0,0,2018,840,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,...,1,0,0,1,0,0,0,0,0,0
11354,12800,1,0,0,0,0,1,1,2016,311,...,0,1,0,0,0,0,0,1,0,0
11355,12801,1,1,1,1,1,1,1,2016,46,...,1,0,0,0,0,0,0,1,0,0
11356,12802,1,1,1,1,1,1,0,2016,48,...,1,0,0,0,0,0,0,1,0,0


In [191]:
# Elimino los outliers de las columnas ASP y ASP_Converted
data_asp_mean = data[data.ASP <= 10].ASP.mean()
outliers =  data.ASP <= 10
data.ASP = data.ASP.where(outliers, data_asp_mean)

data_asp_mean = data[data['ASP_(converted)'] <= 10]['ASP_(converted)'].mean()
outliers =  data['ASP_(converted)'] <= 10
data['ASP_(converted)'] = data['ASP_(converted)'].where(outliers, data_asp_mean)


# Del Tp1

# https://data.oecd.org/conversion/exchange-rates.htm
price_usd_convertion_by_year = {
    'EUR': {2016: 0.940, 2017: 0.923, 2018: 0.848, 2019: 0.893, 2020: 0.860},
    'JPY': {2016: 113.138, 2017: 116.667, 2018: 110.424, 2019: 109.008, 2020: 104.66},
    'AUD': {2016: 1.400, 2017: 1.358, 2018: 1.340, 2019: 1.439, 2020: 1.420},
    'GBP': {2016: 0.770, 2017: 0.808, 2018: 0.750, 2019: 0.784, 2020: 0.770},
    'USD': {2016: 1, 2017: 1, 2018: 1, 2019: 1, 2020: 1}
}

def get_convertion_usd(x):
    currency = x['Total_Amount_Currency']
    year = pd.to_datetime(x['Month']).year
    return price_usd_convertion_by_year[currency][year]
    
data['Total_Amount_USD'] = data['Total_Amount'] / data.apply(lambda x: get_convertion_usd(x), axis=1)

data['Total_Amount_USD'].head()

0    5.609362e+06
1    5.130851e+04
2    8.386560e+04
3    7.421882e+06
4    1.335719e+07
Name: Total_Amount_USD, dtype: float64

In [192]:
# Agrego como feature el promedio de asp
u = data.groupby('Opportunity_ID').ASP.mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)

In [193]:
# Agrego como feature el promedio de asp converted
u = data.groupby('Opportunity_ID')['ASP_(converted)'].mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)

In [194]:
# Agrego como feature el promedio de Total_Amount_USD
u = data.groupby('Opportunity_ID')['Total_Amount_USD'].mean()\
        .reset_index().drop('Opportunity_ID', axis=1)
X = pd.concat([X,u], axis=1, sort=False)
X

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None,ASP,ASP_(converted),Total_Amount_USD
0,0,1,1,1,1,1,1,0,2016,189,...,0,0,1,0,0,0,0,0.5200,0.58817,5.609362e+06
1,1,0,0,1,0,0,1,0,2016,39,...,0,0,1,0,0,0,0,0.5300,0.59948,5.130851e+04
2,2,0,0,1,0,0,1,0,2016,296,...,0,0,0,0,1,0,0,0.4800,0.48000,8.386560e+04
3,3,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,1,0,0,0.5300,0.53000,7.421882e+06
4,4,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,1,0,0,0.5300,0.53000,1.335719e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,...,1,0,0,0,0,0,0,0.5150,0.58251,4.273404e+05
11354,12800,1,0,0,0,0,1,1,2016,311,...,0,0,0,0,1,0,0,0.5300,0.53000,1.066625e+07
11355,12801,1,1,1,1,1,1,1,2016,46,...,0,0,0,0,1,0,0,0.5200,0.58817,1.062819e+05
11356,12802,1,1,1,1,1,1,0,2016,48,...,0,0,0,0,1,0,0,0.6375,0.63750,2.346797e+06


In [195]:
X.isna().sum()

Opportunity_ID                         0
Pricing, Delivery_Terms_Quote_Appr     0
Pricing, Delivery_Terms_Approved       0
pricing_aprobada                       0
Bureaucratic_Code_0_Approval           0
Bureaucratic_Code_0_Approved           0
bureaucratic_aprobada                  0
amount_ids                             0
min_year                               0
active_days                            0
has_source                             0
has_brand                              0
has_product_Type                       0
has_size                               0
has_product_Category_B                 0
has_price                              0
has_currency                           0
Delivery_Quarter                       0
month                                  0
year                                   0
APAC                                   0
Americas                               0
EMEA                                   0
Japan                                  0
Middle East     

In [196]:
total_mean = X.Total_Amount_USD.mean()
total_mean

1274441.2300573422

In [197]:
X = X.replace({np.NaN: total_mean})

## Prediccion

In [198]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358 entries, 0 to 11357
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      11358 non-null  int64  
 1   Pricing, Delivery_Terms_Quote_Appr  11358 non-null  int16  
 2   Pricing, Delivery_Terms_Approved    11358 non-null  int16  
 3   pricing_aprobada                    11358 non-null  int64  
 4   Bureaucratic_Code_0_Approval        11358 non-null  object 
 5   Bureaucratic_Code_0_Approved        11358 non-null  object 
 6   bureaucratic_aprobada               11358 non-null  int64  
 7   amount_ids                          11358 non-null  int8   
 8   min_year                            11358 non-null  int16  
 9   active_days                         11358 non-null  int64  
 10  has_source                          11358 non-null  int8   
 11  has_brand                           11358

In [199]:
X.Bureaucratic_Code_0_Approval = X.Bureaucratic_Code_0_Approval.astype(np.int8)
X.Bureaucratic_Code_0_Approved = X.Bureaucratic_Code_0_Approved.astype(np.int8)

In [200]:
train_ids = train.Opportunity_ID.unique()
trainX = X[X['Opportunity_ID'].isin(train_ids)]
trainX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None,ASP,ASP_(converted),Total_Amount_USD
0,0,1,1,1,1,1,1,0,2016,189,...,0,0,1,0,0,0,0,0.5200,0.58817,5.609362e+06
1,1,0,0,1,0,0,1,0,2016,39,...,0,0,1,0,0,0,0,0.5300,0.59948,5.130851e+04
2,2,0,0,1,0,0,1,0,2016,296,...,0,0,0,0,1,0,0,0.4800,0.48000,8.386560e+04
3,3,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,1,0,0,0.5300,0.53000,7.421882e+06
4,4,1,0,0,1,0,0,0,2018,840,...,0,0,0,0,1,0,0,0.5300,0.53000,1.335719e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11353,12799,1,1,1,0,0,1,0,2016,119,...,1,0,0,0,0,0,0,0.5150,0.58251,4.273404e+05
11354,12800,1,0,0,0,0,1,1,2016,311,...,0,0,0,0,1,0,0,0.5300,0.53000,1.066625e+07
11355,12801,1,1,1,1,1,1,1,2016,46,...,0,0,0,0,1,0,0,0.5200,0.58817,1.062819e+05
11356,12802,1,1,1,1,1,1,0,2016,48,...,0,0,0,0,1,0,0,0.6375,0.63750,2.346797e+06


In [201]:
y = train.groupby('Opportunity_ID')['Stage'].min()
y

Opportunity_ID
0        0
1        1
2        1
3        0
4        0
        ..
12799    1
12800    0
12801    1
12802    0
12803    0
Name: Stage, Length: 9791, dtype: int64

In [202]:
(y.index != trainX.Opportunity_ID).sum()

0

In [203]:
test_ids = test.Opportunity_ID.unique()
testX = X[X['Opportunity_ID'].isin(test_ids)]
testX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None,ASP,ASP_(converted),Total_Amount_USD
9348,10689,1,1,1,1,1,1,1,2019,0,...,1,0,0,0,0,0,0,0.306667,0.346870,137147.816349
9349,10690,1,1,1,1,1,1,1,2019,13,...,1,0,0,0,0,0,0,0.332000,0.375524,169716.349384
9350,10691,1,1,1,0,0,1,0,2019,4,...,0,0,1,0,0,0,0,0.425000,0.425000,21037.500000
9351,10692,1,1,1,1,0,0,1,2019,4,...,1,0,0,0,0,0,0,0.425000,0.425000,361517.750000
9352,10693,0,0,1,0,0,1,0,2019,1,...,1,0,0,0,0,0,0,0.390000,0.390000,5752.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10927,12364,1,1,1,0,0,1,1,2019,0,...,1,0,0,0,0,0,0,0.300000,0.300000,73875.000000
10928,12365,1,1,1,0,0,1,0,2019,1,...,0,0,1,0,0,0,0,0.410000,0.410000,45054.900000
10929,12366,1,1,1,1,1,1,0,2019,1,...,0,0,1,0,0,0,0,0.410000,0.410000,100122.000000
10930,12367,1,1,1,0,0,1,0,2019,0,...,0,0,1,0,0,0,0,0.500000,0.500000,143220.000000


In [204]:
testX.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1567 entries, 9348 to 10931
Data columns (total 42 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Opportunity_ID                      1567 non-null   int64  
 1   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   int16  
 2   Pricing, Delivery_Terms_Approved    1567 non-null   int16  
 3   pricing_aprobada                    1567 non-null   int64  
 4   Bureaucratic_Code_0_Approval        1567 non-null   int8   
 5   Bureaucratic_Code_0_Approved        1567 non-null   int8   
 6   bureaucratic_aprobada               1567 non-null   int64  
 7   amount_ids                          1567 non-null   int8   
 8   min_year                            1567 non-null   int16  
 9   active_days                         1567 non-null   int64  
 10  has_source                          1567 non-null   int8   
 11  has_brand                           156

In [237]:
testX

,Opportunity_ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",pricing_aprobada,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,bureaucratic_aprobada,amount_ids,min_year,active_days,...,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_4,Account_Type_5,Account_Type_6,None,ASP,ASP_(converted),Total_Amount_USD
9348,10689,1,1,1,1,1,1,1,2019,0,...,1,0,0,0,0,0,0,0.306667,0.346870,137147.816349
9349,10690,1,1,1,1,1,1,1,2019,13,...,1,0,0,0,0,0,0,0.332000,0.375524,169716.349384
9350,10691,1,1,1,0,0,1,0,2019,4,...,0,0,1,0,0,0,0,0.425000,0.425000,21037.500000
9351,10692,1,1,1,1,0,0,1,2019,4,...,1,0,0,0,0,0,0,0.425000,0.425000,361517.750000
9352,10693,0,0,1,0,0,1,0,2019,1,...,1,0,0,0,0,0,0,0.390000,0.390000,5752.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10927,12364,1,1,1,0,0,1,1,2019,0,...,1,0,0,0,0,0,0,0.300000,0.300000,73875.000000
10928,12365,1,1,1,0,0,1,0,2019,1,...,0,0,1,0,0,0,0,0.410000,0.410000,45054.900000
10929,12366,1,1,1,1,1,1,0,2019,1,...,0,0,1,0,0,0,0,0.410000,0.410000,100122.000000
10930,12367,1,1,1,0,0,1,0,2019,0,...,0,0,1,0,0,0,0,0.500000,0.500000,143220.000000


In [205]:
X = trainX.drop(['Opportunity_ID'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# normalizamos los features

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [253]:
len(X_train[0])

41

In [259]:
len(X_train)

9791

In [279]:
# predicción

input_shape = (len(X_train[0]), 1, 1)

model = Sequential()

model.add(Dense(8, activation='relu', input_shape=(41,)))

model.add(Dense(8, activation='relu'))

model.add(Dense(1, activation='softmax'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=8, batch_size=1, verbose=1)

Epoch 1/8
9791/9791 [==============================] - 5s 542us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 2/8
9791/9791 [==============================] - 6s 591us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 3/8
9791/9791 [==============================] - 6s 567us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 4/8
9791/9791 [==============================] - 6s 620us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 5/8
9791/9791 [==============================] - 6s 582us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 6/8
9791/9791 [==============================] - 6s 595us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 7/8
9791/9791 [==============================] - 7s 701us/step - loss: 7.3500 - accuracy: 0.5180
Epoch 8/8
9791/9791 [==============================] - 6s 593us/step - loss: 7.3500 - accuracy: 0.5180


In [280]:
# predicción

input_shape = (len(X_train[0]), 1, 1)

model = Sequential()

model.add(Dense(8, activation='relu', input_shape=(41,)))

model.add(Dense(8, activation='relu'))

model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train,epochs=8, batch_size=1, verbose=1)

Epoch 1/8
9791/9791 [==============================] - 5s 474us/step - loss: 0.5144 - accuracy: 0.7399
Epoch 2/8
9791/9791 [==============================] - 5s 497us/step - loss: 0.4744 - accuracy: 0.7691
Epoch 3/8
9791/9791 [==============================] - 5s 507us/step - loss: 0.4638 - accuracy: 0.7743
Epoch 4/8
9791/9791 [==============================] - 5s 532us/step - loss: 0.4472 - accuracy: 0.7795
Epoch 5/8
9791/9791 [==============================] - 5s 461us/step - loss: 0.4412 - accuracy: 0.7868
Epoch 6/8
9791/9791 [==============================] - 5s 460us/step - loss: 0.4360 - accuracy: 0.7880
Epoch 7/8
9791/9791 [==============================] - 4s 430us/step - loss: 0.4322 - accuracy: 0.78390s - loss: 0.4
Epoch 8/8
9791/9791 [==============================] - 5s 481us/step - loss: 0.4293 - accuracy: 0.7891


In [281]:
y_pred = model.predict(X_test)

score = model.evaluate(X_test, y_test,verbose=1)

print(score)

62/62 [==============================] - 0s 383us/step - loss: 0.4048 - accuracy: 0.7979
[0.4047631025314331, 0.7978560328483582]


In [282]:
y_pred


array([[0.5764793],
       [0.2784416],
       [0.3231619],
       ...,
       [0.920202 ],
       [0.0335986],
       [0.4610618]], dtype=float32)

In [208]:
# Calculo el error
ll = log_loss(y_test, y_pred, normalize=True)
print("Log Loss: %f" % (ll))

Log Loss: 0.440485


In [209]:
# Predicción para kaggle

# normalizamos los features

scaler = StandardScaler().fit(X)

X_train = scaler.transform(X)
y_train = y
# entrenamos
model.fit(X_train, y_train,epochs=8, batch_size=1, verbose=1)

Epoch 1/8
9791/9791 [==============================] - 5s 481us/step - loss: 0.4354 - accuracy: 0.7898
Epoch 2/8
9791/9791 [==============================] - 5s 471us/step - loss: 0.4335 - accuracy: 0.78790s - loss: 0.4326 - accuracy: 
Epoch 3/8
9791/9791 [==============================] - 4s 455us/step - loss: 0.4308 - accuracy: 0.7902
Epoch 4/8
9791/9791 [==============================] - 5s 461us/step - loss: 0.4291 - accuracy: 0.79210s - l
Epoch 5/8
9791/9791 [==============================] - 4s 431us/step - loss: 0.4255 - accuracy: 0.7932
Epoch 6/8
9791/9791 [==============================] - 5s 504us/step - loss: 0.4240 - accuracy: 0.7914
Epoch 7/8
9791/9791 [==============================] - 5s 475us/step - loss: 0.4233 - accuracy: 0.7923
Epoch 8/8
9791/9791 [==============================] - 5s 533us/step - loss: 0.4204 - accuracy: 0.7929


In [211]:
preds = model.predict(testX.drop(['Opportunity_ID'], axis=1))

In [227]:
preds.shape

(1567, 1)

In [228]:
preds

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [213]:
prediccion = pd.DataFrame({'Opportunity_ID':testX.Opportunity_ID, 'Target':preds})
prediccion

Exception: Data must be 1-dimensional

In [ ]:
prediccion.to_csv('prediccion_keras', index=False)